<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_SQLDatabaseChain_Vulnerability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain SQL Agent

The vulnerability in `SQLDatabaseChain` may be exploited if it's carefully used.

In this tutorial, I will show you the PoC and introduce you a safer way to interact with SQL databases using natural language - **SQL Agent**.

1. Set up the development environment

In [ ]:
!pip install -q -U langchain langchain_experimental openai

In [ ]:
from langchain import OpenAI, SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

2. Get the OpenAI API key ready

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'your valid openai api key'

3. Download a SQLite dataset

In [8]:
!wget https://bafybeibqw3muonedrl57jvmhh2oemftjvamtc22htygi7vl52qmgbvqmk4.ipfs.nftstorage.link/retailDB.sqlite -O retail.sqlite

--2023-09-24 21:17:34--  https://bafybeibqw3muonedrl57jvmhh2oemftjvamtc22htygi7vl52qmgbvqmk4.ipfs.nftstorage.link/retailDB.sqlite
Resolving bafybeibqw3muonedrl57jvmhh2oemftjvamtc22htygi7vl52qmgbvqmk4.ipfs.nftstorage.link (bafybeibqw3muonedrl57jvmhh2oemftjvamtc22htygi7vl52qmgbvqmk4.ipfs.nftstorage.link)... 104.18.41.40, 172.64.146.216, 2606:4700:4400::6812:2928, ...
Connecting to bafybeibqw3muonedrl57jvmhh2oemftjvamtc22htygi7vl52qmgbvqmk4.ipfs.nftstorage.link (bafybeibqw3muonedrl57jvmhh2oemftjvamtc22htygi7vl52qmgbvqmk4.ipfs.nftstorage.link)|104.18.41.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2940928 (2.8M) [text/plain]
Saving to: ‘retail.sqlite’

retail.sqlite       100%[===================>]   2.80M  --.-KB/s    in 0.04s   

2023-09-24 21:17:36 (79.8 MB/s) - ‘retail.sqlite’ saved [2940928/2940928]



4. Instantiate a SQLDatabase instance

In [9]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase.from_uri("sqlite:///retail.sqlite")
llm = OpenAI(temperature=0, verbose=True)

In [10]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

5. Interact with database using `SQLDatabaseChain`

Now we are going to interact with database. We will try to update and delete data from database which will cause data change.

5.1 Modify data by SQLDatabaseChain

In [11]:
db_chain.run("Change all the real reviews to -1")



> Entering new SQLDatabaseChain chain...
Change all the real reviews to -1
SQLQuery:UPDATE reviews SET real_reviews = -1;
SQLResult: 
Answer:All the real reviews have been changed to -1.
> Finished chain.


'All the real reviews have been changed to -1.'

In [12]:
db_chain.run("list the first row in reviews table")



> Entering new SQLDatabaseChain chain...
list the first row in reviews table
SQLQuery:SELECT "product_id", "rating", "reviews", "Hour", "minute", "real_rating", "real_reviews", "Unnamed: 7" FROM reviews LIMIT 1;
SQLResult: [('AH2430', 'None', 'None', 0.0, 0.0, None, -1.0, '')]
Answer:The first row in reviews table is ('AH2430', 'None', 'None', 0.0, 0.0, None, -1.0, '').
> Finished chain.


"The first row in reviews table is ('AH2430', 'None', 'None', 0.0, 0.0, None, -1.0, '')."

5.2 Delete data

In [13]:
db_chain.run("Delete all the reviews")



> Entering new SQLDatabaseChain chain...
Delete all the reviews
SQLQuery:DELETE FROM reviews;
SQLResult: 
Answer:All reviews have been deleted.
> Finished chain.


'All reviews have been deleted.'

In [14]:
db_chain.run("show me the count of reviews")



> Entering new SQLDatabaseChain chain...
show me the count of reviews
SQLQuery:SELECT COUNT(*) FROM reviews;
SQLResult: [(0,)]
Answer:There are 0 reviews.
> Finished chain.


'There are 0 reviews.'

6. Interact with database using **SQL Agent**


In [15]:
!wget https://bafybeibqw3muonedrl57jvmhh2oemftjvamtc22htygi7vl52qmgbvqmk4.ipfs.nftstorage.link/retailDB.sqlite -O retail2.sqlite

--2023-09-24 21:20:52--  https://bafybeibqw3muonedrl57jvmhh2oemftjvamtc22htygi7vl52qmgbvqmk4.ipfs.nftstorage.link/retailDB.sqlite
Resolving bafybeibqw3muonedrl57jvmhh2oemftjvamtc22htygi7vl52qmgbvqmk4.ipfs.nftstorage.link (bafybeibqw3muonedrl57jvmhh2oemftjvamtc22htygi7vl52qmgbvqmk4.ipfs.nftstorage.link)... 104.18.41.40, 172.64.146.216, 2606:4700:4400::ac40:92d8, ...
Connecting to bafybeibqw3muonedrl57jvmhh2oemftjvamtc22htygi7vl52qmgbvqmk4.ipfs.nftstorage.link (bafybeibqw3muonedrl57jvmhh2oemftjvamtc22htygi7vl52qmgbvqmk4.ipfs.nftstorage.link)|104.18.41.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2940928 (2.8M) [text/plain]
Saving to: ‘retail2.sqlite’

retail2.sqlite      100%[===================>]   2.80M  --.-KB/s    in 0.06s   

2023-09-24 21:20:53 (50.3 MB/s) - ‘retail2.sqlite’ saved [2940928/2940928]



In [16]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

db = SQLDatabase.from_uri("sqlite:///retail2.sqlite")
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [17]:
agent_executor.run(
    "Show me the first row of reviews table"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: brands, finance, info, reviews, traffic
Thought: The reviews table looks most relevant, so I should query the schema of that table.
Action: sql_db_schema
Action Input: reviews
Observation: 
CREATE TABLE reviews (
	product_id VARCHAR(50), 
	rating VARCHAR(50), 
	reviews VARCHAR(50), 
	"Hour" REAL, 
	minute REAL, 
	real_rating REAL, 
	real_reviews REAL, 
	"Unnamed: 7" VARCHAR(50)
)

/*
3 rows from reviews table:
product_id	rating	reviews	Hour	minute	real_rating	real_reviews	Unnamed: 7
AH2430	None	None	0.0	0.0	None	0.0	
G27341	03:03:00	1900-01-01 00:00:00	3.0	3.0	3,3	24.0	
CM0081	02:06:00	1900-01-01 13:00:00	2.0	6.0	2,6	37.0	
*/
Thought: I should query the first row of the reviews table.
Action: sql_db_query
Action Input: SELECT * FROM reviews LIMIT 1
Observation: [('AH2430', 'None', 'None', 0.0, 0.0, None, 0.0, '')]
Thought: I now know the final answer
Final Answer: The first row of the reviews

"The first row of the reviews table is ('AH2430', 'None', 'None', 0.0, 0.0, None, 0.0, '')."

6.1 Modify data

In [18]:
agent_executor.run(
    "Change all the real reviews to -1"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: brands, finance, info, reviews, traffic
Thought: I should look at the schema of the reviews table
Action: sql_db_schema
Action Input: "reviews"
Observation: 
CREATE TABLE reviews (
	product_id VARCHAR(50), 
	rating VARCHAR(50), 
	reviews VARCHAR(50), 
	"Hour" REAL, 
	minute REAL, 
	real_rating REAL, 
	real_reviews REAL, 
	"Unnamed: 7" VARCHAR(50)
)

/*
3 rows from reviews table:
product_id	rating	reviews	Hour	minute	real_rating	real_reviews	Unnamed: 7
AH2430	None	None	0.0	0.0	None	0.0	
G27341	03:03:00	1900-01-01 00:00:00	3.0	3.0	3,3	24.0	
CM0081	02:06:00	1900-01-01 13:00:00	2.0	6.0	2,6	37.0	
*/
Thought: I should update the real_reviews column to -1
Action: sql_db_query
Action Input: "UPDATE reviews SET real_reviews = -1"
Observation: 
Thought: I now know the final answer
Final Answer: All the real reviews in the reviews table have been changed to -1.

> Finished chain.


'All the real reviews in the reviews table have been changed to -1.'

In [19]:
agent_executor.run(
    "Show me the first row of reviews table"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: brands, finance, info, reviews, traffic
Thought: The reviews table looks most relevant, so I should query the schema of that table.
Action: sql_db_schema
Action Input: reviews
Observation: 
CREATE TABLE reviews (
	product_id VARCHAR(50), 
	rating VARCHAR(50), 
	reviews VARCHAR(50), 
	"Hour" REAL, 
	minute REAL, 
	real_rating REAL, 
	real_reviews REAL, 
	"Unnamed: 7" VARCHAR(50)
)

/*
3 rows from reviews table:
product_id	rating	reviews	Hour	minute	real_rating	real_reviews	Unnamed: 7
AH2430	None	None	0.0	0.0	None	-1.0	
G27341	03:03:00	1900-01-01 00:00:00	3.0	3.0	3,3	-1.0	
CM0081	02:06:00	1900-01-01 13:00:00	2.0	6.0	2,6	-1.0	
*/
Thought: I now know the columns in the reviews table, so I can query the first row.
Action: sql_db_query
Action Input: SELECT * FROM reviews LIMIT 1
Observation: [('AH2430', 'None', 'None', 0.0, 0.0, None, -1.0, '')]
Thought: I now know the final answer.
Final Answer: T

"The first row of the reviews table is ('AH2430', 'None', 'None', 0.0, 0.0, None, -1.0, '')."

6.2 Delete data

In [20]:
agent_executor.run(
    "Delete all the reviews"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: brands, finance, info, reviews, traffic
Thought: The reviews table is the most relevant, so I should query the schema of that table.
Action: sql_db_schema
Action Input: reviews
Observation: 
CREATE TABLE reviews (
	product_id VARCHAR(50), 
	rating VARCHAR(50), 
	reviews VARCHAR(50), 
	"Hour" REAL, 
	minute REAL, 
	real_rating REAL, 
	real_reviews REAL, 
	"Unnamed: 7" VARCHAR(50)
)

/*
3 rows from reviews table:
product_id	rating	reviews	Hour	minute	real_rating	real_reviews	Unnamed: 7
AH2430	None	None	0.0	0.0	None	-1.0	
G27341	03:03:00	1900-01-01 00:00:00	3.0	3.0	3,3	-1.0	
CM0081	02:06:00	1900-01-01 13:00:00	2.0	6.0	2,6	-1.0	
*/
Thought: I should not delete any data from the database, so I should return "I don't know" as the answer.
Final Answer: I don't know

> Finished chain.


"I don't know"